# 동적 페이지 크롤링하기(3)

# 네이버 웹툰 댓글 가져오기

해당 웹툰의 해당 화 페이지로 requests.get 했더니,  
안가져와지네?? -> iframe  
iframe_url로 requests.get했더니, 
안가져와지네?? -> javascript  
network tab에서 preview 보면서 requests_url을 찾아낸다.

In [2]:
import json 
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

### 하나의 웹툰, 하나의 화 전체 댓글 가져오기

In [3]:
webtoon = {
        'index' : [],
        '댓글' : [],
        '공감수' : [],
        '비공감수' :[]
        }

In [13]:
# 전체 댓글의 맨 뒤페이지 번호 가져오기
url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'



params ={
        'ticket' : 'comic',  
        'templateId' : 'webtoon',  
        'pool' : 'cbox3',  
        '_callback' : '', # jQuery112409091552467251651_1615854232166 -> ''  
        'lang' : 'ko',  
        'country' : 'KR',  
        'objectId' : '650305_298', # 웹툰의 고유 아이디_해당화 인덱스  
        'pageSize' : 15,  
        'indexSize' : 10,  
        'listType' : 'OBJECT',  
        'pageType' : 'default',  
        'page' : 1, # 전체댓글의 페이지 번호 
        'refresh' : 'true',  
        'sort' : 'NEW' # 전체댓글 URL의 고유 쿼리 
        }

headers = {
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'

}

resp = requests.get(url, params=params, headers=headers)
 
resp

<Response [200]>

In [14]:
resp.text # _callback({ ~~~~~ }); -> { ~~~~~ }만 뽑아내야한다. 

'_callback({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"650305_298","categoryId":"*","templateId":"default_v2","commentNo":"434980817","parentCommentNo":"434980817","replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1646806958148,"contents":"추이와 무케 장면에서 무커 팔로만 위로해주고 무케도 꼭 잡고있는 디테일한 장면 정주행중인데 너무 마음 아프네요 ㅠ","userIdNo":"5PV3u","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"225","userProfileImage":"https://profile-phinf.pstatic.net/36750/006b8dcb3138da0c90cf379e6ee1bd7c32cc8fb120bb5f69807cb497bb3e4292.jpg","profileType":"naver","modTime":"2022-03-09T15:22:38+0900","modTimeGmt":"2022-03-09T06:22:38+0000","regTime":"2022-03-09T15:22:38+0900","regTimeG

In [ ]:
resp.text[10:-2]

In [ ]:
dic_resp = json.loads(resp.text[10:-2]) # resp.text[10:-2] -> { ~~~~~ } 
dic_resp

In [ ]:
totalPages = dic_resp['result']['pageModel']['totalPages']
totalPages

In [ ]:
# best 댓글
# url  :  'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery112408122866196452874_1612750131447&lang=ko&country=KR&objectId=650305_298&pageSize=15&indexSize=10&listType=OBJECT&pageType=default&page=1&initialize=true&useAltSort=true&replyPageSize=10&_=1612750131448'

# 전체 댓글
url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'

for page in range(1, totalPages+1):
    params ={
            'ticket' : 'comic',  
            'templateId' : 'webtoon',  
            'pool' : 'cbox3',  
            '_callback' : '',  
            'lang' : 'ko',  
            'country' : 'KR',  
            'objectId' : '650305_298', # 웹툰의 고유 아이디_해당화 인덱스  
            'pageSize' : 15,  
            'indexSize' : 10,  
            'listType' : 'OBJECT',  
            'pageType' : 'default',  
            'page' : page, # 전체댓글의 페이지 번호 
            'refresh' : 'true',  
            'sort' : 'NEW', # 전체댓글 URL의 고유 쿼리 
            '_' : 1612753212403
            }

    headers = {
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=650305&no=298',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }

    resp = requests.get(url, params=params, headers=headers)




    dic_resp = json.loads(resp.text[10:-2])
    dic_resp
    # result commentList : [] 'antipathyCount', 'contents', 'commentNo', 'sympathyCount'

    # webtoon = {
    #         'index' : [],
    #         '댓글' : [],
    #         '공감수' : [],
    #         '비공감수' :[]
    #         }

    commentList = dic_resp['result']['commentList']

    for comment in commentList:
        webtoon['index'].append(comment['commentNo'])
        webtoon['댓글'].append(comment['contents'])
        webtoon['공감수'].append(comment['sympathyCount'])
        webtoon['비공감수'].append(comment['antipathyCount'])


In [ ]:
df_webtoon = pd.DataFrame(webtoon)
df_webtoon

,index,댓글,공감수,비공감수
0,409153953,작가님 앞으론 중국산 양산형 게임과는 협업하지 않으셨으면 좋겠습니다.,0,0
1,409153842,무케 사랑해 엉엉,0,0
2,409152830,시 대사 잘못읽은줄 알았네ㅋㅋ,0,0
3,409151036,야..야!! 뒤에봐 뒤에!!!,0,0
4,409150589,어어...밀지마라,0,0
...,...,...,...,...
2511,407446644,4,0,1
2512,407446633,으may,0,1
2513,407446621,1,0,1
2514,407446574,1,0,1


In [ ]:
df_webtoon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   2516 non-null   int64 
 1   댓글      2516 non-null   object
 2   공감수     2516 non-null   int64 
 3   비공감수    2516 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 78.8+ KB


In [ ]:
df_webtoon[df_webtoon['공감수'] >= 100]

,index,댓글,공감수,비공감수
2095,407452914,"하,....무케 대사 넘 눈물난다 저 어린애가...뭘알겠어\n그리고 추이도....그...",3411,49
2098,407452764,추이가 무케 볼때 염주는 일부러 푼건가...아님 그냥 원래 모습으로 표현된걸까 참 ...,2649,47
2201,407450943,이번화 요약\n1기억찾은 추이형님이 폐하 구다국으로 데려다드림\n2가는 길에 무케 ...,4926,101
2235,407450375,폐하를 추이가 데리고 온거잖아요...데려다 놓고 무케랑 작별인사하고 이제 붉은 산으...,2674,61
2236,407450364,추이 말하지않아도 너의 감정들이 다보인다ㅠㅠㅠㅠㅠㅠㅠ 진짜 우째.......,2905,50
2238,407450316,"‘아저씨한테 아빠냄새 난다’고 하고, 나중엔 ‘아빠한테 아저씨 냄새 난다’고 하네..ㅠㅜ",5505,53
2243,407450260,매 화마다 감동적이고 격동적이길 바라지 마라 \n작가님 마음가는대로 그릴 수 있도록...,3475,204
2244,407450259,무케한테 팔을 내주고 있는 추이는 꿈속 상상이 아니라 진짜 추이구나. 무커가 한 대...,4625,39
2286,407449707,"중간에 시 미친거아니냐ㅋㅋㅋ \n너희 대장""님""을 이령""놈""이 데리고 있다네ㅋㅋ",10459,76
2302,407449473,와 추이 깨어나고 구다국왕 모셔다놓고 무케 한번 보고 바로 붉은 산 직행했구나,11444,45


### 원하는 웹툰을 입력받아, 해당 웹툰의 전체 댓글 가져오기

In [ ]:
MyWebtoon = input('어떤 웹툰의 댓글을 가져올까요?')
print('네 {}의 댓글을 수집하겠습니다.'.format(MyWebtoon))

어떤 웹툰의 댓글을 가져올까요?여신강림
네 여신강림의 댓글을 수집하겠습니다.


In [ ]:
url = 'https://comic.naver.com/webtoon/weekday.nhn'
resp = requests.get(url)
resp

<Response [200]>

In [ ]:
soup = BeautifulSoup(resp.content, 'lxml')
MyURL = 'https://comic.naver.com' + soup.select("a[title={}]".format(MyWebtoon))[0]['href']
title_id = MyURL.split('&')[0].split('=')[1]
title_id

'758037'

In [ ]:
MyURL

'https://comic.naver.com/webtoon/list.nhn?titleId=758037&weekday=mon'

In [ ]:
resp = requests.get(MyURL)
soup = BeautifulSoup(resp.content, 'lxml')
currentNo = soup.select('td.title a')[0]['href'].split('no=')[1].split('&')[0]
currentNo

'15'

In [ ]:
import json
from tqdm.notebook import tqdm

for no in tqdm(range(1, int(currentNo)+1), desc=MyWebtoon):
    # 끝페이지 가져오기
    url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery112408459589214855283_1612831780064&lang=ko&country=KR&objectId={titleId}_{no}&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=1&refresh=true&sort=new&_=1612831780066'.format(titleId=title_id, no=no)

    headers = {
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=758037&no=15',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    resp = requests.get(url, headers=headers)

    dic_resp = json.loads(resp.text[42:-2])

    last_pg = dic_resp['result']['pageModel']['totalPages']

    # 전체댓글 가져오기
    for pg in tqdm(range(1, int(last_pg)+1), desc=str(no)+'화'):
        url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery112408459589214855283_1612831780064&lang=ko&country=KR&objectId={titleId}_{no}&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=1&refresh=true&sort=new&_=1612831780066'.format(titleId=title_id, no=no)

        headers = {
            'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=758037&no=15',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
        }
        resp = requests.get(url, headers=headers)

        dic_resp = json.loads(resp.text[42:-2])

        for comment in dic_resp['result']['commentList']:
            # print(comment['contents'])
            pass

